In [1]:
from ipywidgets import interact
import ipywidgets as widgets
import asyncio
import json
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import subprocess
from IPython.display import FileLink
import os
def display_image(path):
    img = mpimg.imread(path)
    plt.imshow(img)
    plt.axis('off')  # Turn off the axis numbers and ticks
    plt.show()
    
def wait_for_change(func_button):
    future = asyncio.Future()

    def evaluate(button):
        future.set_result(button.description)
        func_button.on_click(
            evaluate, remove=True
        )  # we need to free up the binding to getvalue to avoid an InvalidState error buttons don't support unobserve, so use `remove=True`

    func_button.on_click(evaluate)
    return future

In [2]:

original_file = "annotate_0_10000.json"
START = 0
FINISH = 10000
f = open(original_file, "r")
content = f.read()
list_file = json.loads(content)

In [3]:
out = widgets.Output()  # Create an Output widget to capture and display items


def display_data_point(i, dic_data):
    if dic_data.get('if_annotate', False):
        if i + 1 < len(list_file[START:FINISH]):
            display_data_point(i + 1, list_file[START + i + 1])
        return
    with out:
        figure_path = dic_data['figure_path']
        caption = dic_data['caption']
        print(figure_path)
        display_image(figure_path)
        caption_display = widgets.Label(value="Figure Caption:" + caption)
        display(caption_display)
        
        #path_display = widgets.Label(value="Figure Path:" + figure_path)
        file_link = FileLink(os.path.abspath(figure_path))

        # Display the link
        display(file_link)
        custom_layout = widgets.Layout(width='40%', height='500px')
        prompt = "This is a data visualization figure from an academic paper with the caption of " + caption + "\n Please write Python code using matplotlib to draw the exact same plot as this one and save it as a png file with 300dpi."
        subprocess.run(['clip.exe'],
                       input=prompt.encode('utf-16'),
                       check=True)
        code_input = widgets.Textarea(value='',
                                      placeholder='Type your code here',
                                      description='Code:',
                                      disabled=False,
                                      layout=custom_layout)
        code_input_2 = widgets.Textarea(value='',
                                      placeholder="Let's think step-by-step",
                                      description='Code:',
                                      disabled=False,
                                      layout=custom_layout)
        button = widgets.Button(description="Submit",
                                disabled=False,
                                button_style="")
        visualization_types = [
            "Bar Chart",
            "Line Chart",
            "Pie Chart",
            "Histogram",
            "Scatter Plot",
            "Box Plot",
            "Heat Map",
            "Others"
        ]
        figure_type = widgets.RadioButtons(
                options=visualization_types,
                value=None,
                description="Please select the figure type",
                disabled=False,
            )
        is_data_visual = widgets.RadioButtons(
                options=["Yes", "No"],
                value=None,
                description="Is this figure a data visualization plot?",
                disabled=False,
            )
        Box = widgets.VBox([is_data_visual, figure_type, code_input, code_input_2, button])
        display(Box)

        def on_submit(btn):
            is_visual = is_data_visual.value
            code = code_input.value
            code_cot = code_input_2.value
            type = figure_type.value
            dic_data["code"] = code
            dic_data["code_cot"] = code_cot
            dic_data["is_data_visual"] = is_visual
            dic_data["type"] = type
            dic_data["if_annotate"] = True
            list_file[i] = dic_data
            with open(
                    "annotate" + "_" + str(START) + "_" + str(FINISH) +
                    ".json", "w") as file:
                json.dump(list_file, file, indent=4)
            out.clear_output(wait=True)
            if i + 1 < len(list_file[START:FINISH]):
                display_data_point(i + 1, list_file[START + i + 1])

        button.on_click(on_submit)


display(out)  # Display the Output widget initially

# Start the process by displaying the first data point
display_data_point(0, list_file[START])

Output()